<a href="https://colab.research.google.com/github/XavierCarrera/Accomodation-reservation-prediction/blob/main/Reserve_Accomodation_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction and Problem Definition

In [93]:
import numpy as np
import pandas as pd
from itertools import islice 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [94]:
x = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/db/train_data.txt", sep=" ", header = None)
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/db/DataAcomodacion.csv", encoding="latin-1")

In [72]:
test

,id,duracion_estadia,genero,edad,niños,codigo_destino,tipo_acomodacion
0,1013321,2,F,38.0,0.0,US,NaN
1,1009487,6,M,37.0,0.0,ES,NaN
2,1002727,1,M,37.0,1.0,ES,NaN
3,1001135,5,F,50.0,0.0,UK,NaN
4,1000796,4,F,42.0,1.0,AR,NaN
...,...,...,...,...,...,...,...
4224,1007287,3,M,NaN,0.0,ES,NaN
4225,1009717,4,F,35.0,NaN,NL,NaN
4226,1012540,7,F,NaN,1.0,PE,NaN
4227,1000617,4,M,48.0,0.0,UK,NaN


In [95]:
test.drop(["id"], axis=1)

,duracion_estadia,genero,edad,niños,codigo_destino,tipo_acomodacion
0,2,F,38.0,0.0,US,NaN
1,6,M,37.0,0.0,ES,NaN
2,1,M,37.0,1.0,ES,NaN
3,5,F,50.0,0.0,UK,NaN
4,4,F,42.0,1.0,AR,NaN
...,...,...,...,...,...,...
4224,3,M,NaN,0.0,ES,NaN
4225,4,F,35.0,NaN,NL,NaN
4226,7,F,NaN,1.0,PE,NaN
4227,4,M,48.0,0.0,UK,NaN


In [96]:
x

,0,1
0,[REG],Registro 1
1,[REG],1010286
2,[REG],7
3,[REG],F
4,[REG],40
...,...,...
78939,[REG],F
78940,[REG],49
78941,[REG],1
78942,[REG],IT


# Data Engineering

In [97]:
x.drop(x.columns[0], axis=1, inplace=True)
#x = x.T
x

,1
0,Registro 1
1,1010286
2,7
3,F
4,40
...,...
78939,F
78940,49
78941,1
78942,IT


In [102]:
arrCollection = x[1].str.extract('(Registro.*)').ffill()[0].to_numpy()
df = x.set_index([x.groupby(arrCollection)[1].cumcount()-1,
                     arrCollection])[1].unstack().iloc[1:,:]

train = df.T
train

,0,1,2,3,4,5,6
Registro 1,1010286,7,F,40,0,COL,AirBnB
Registro 10,1006149,5,F,31,NaN,AR,AirBnB
Registro 100,1007317,5,M,42,0,PE,AirBnB
Registro 1000,1007185,6,M,46,1,IT,Hotel
Registro 1001,1002065,4,F,43,0,COL,AirBnB
...,...,...,...,...,...,...,...
Registro 995,1011356,4,F,33,1,AR,Hotel
Registro 996,1008195,4,F,37,1,ES,Hotel
Registro 997,1005338,6,M,38,0,AR,Hotel
Registro 998,1014092,7,M,42,1,AR,Hotel


In [103]:
test.columns

Index(['id', 'duracion_estadia', 'genero', 'edad', 'niños', 'codigo_destino',
       'tipo_acomodacion'],
      dtype='object')